In [1]:
import escher
from escher import Builder
import cobra
from time import sleep

In [13]:
# Load the model into cobra
model = cobra.io.load_json_model('e_coli_core.json')

In [17]:
# Check what the default medium is
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

In [18]:
# Run FBA for the default medium
glc_10_solution = model.optimize()

In [21]:
# Make a map with the solution for the default medium
glc_10_map = Builder(
    map_name = 'e_coli_core.Core metabolism',
    reaction_data = glc_10_solution.fluxes
)

In [22]:
# View the map
glc_10_map

Builder(reaction_data={'PFK': 7.477381962160286, 'PFL': 0.0, 'PGI': 4.860861146496819, 'PGK': -16.023526143167…

In [23]:
# Define medium with just a little bit of acetate available
# Note, this does not set the flux, just the limits on it
glc_ace_medium = {
    'EX_co2_e': 1000.0,
    'EX_glc__D_e': 10.0,
    'EX_ac_e': 1,
    'EX_h_e': 1000.0,
    'EX_h2o_e': 1000.0,
    'EX_nh4_e': 1000.0,
    'EX_o2_e': 1000.0,
    'EX_pi_e': 1000.0
}

In [24]:
# Run FBA for the glucose + acetate medium
model.medium = glc_ace_medium
ace_1_solution = model.optimize()

In [25]:
# Make a map with the solution for the defulat medium
ace_1_map = Builder(
    map_name = 'e_coli_core.Core metabolism',
    reaction_data = ace_1_solution.fluxes
)

In [26]:
ace_1_map

Builder(reaction_data={'PFK': 7.506344253575243, 'PFL': 0.0, 'PGI': 5.011434394904456, 'PGK': -16.014365066565…

In [28]:
# Get a list of the reactions that are different between the two solutions, with the difference between them
diff_reactions = {}
for reaction in model.reactions:
    if abs(glc_10_solution.fluxes[reaction.id] - ace_1_solution.fluxes[reaction.id]) > 1e-6:
        diff_reactions[reaction.id] = ace_1_solution.fluxes[reaction.id] - glc_10_solution.fluxes[reaction.id]

In [31]:
# Print the reaction ID, the reaction name, the flux in the glucose medium, and the flux in the glucose + acetate medium in the order of the difference between the two
for reaction in sorted(diff_reactions, key = lambda x: abs(diff_reactions[x])):
    print(reaction, model.reactions.get_by_id(reaction).name, glc_10_solution.fluxes[reaction], ace_1_solution.fluxes[reaction])

GLNS Glutamine synthetase 0.22346172933182762 0.22932139824385633
PGK Phosphoglycerate kinase -16.02352614316761 -16.014365066565162
GAPD Glyceraldehyde-3-phosphate dehydrogenase 16.023526143167608 16.01436506656516
BIOMASS_Ecoli_core_w_GAM Biomass Objective Function with GAM 0.8739215069684305 0.8968376935622071
PFK Phosphofructokinase 7.477381962160286 7.506344253575243
TPI Triose-phosphate isomerase 7.477381962160286 7.506344253575243
FBA Fructose-bisphosphate aldolase 7.477381962160286 7.506344253575243
RPI Ribose-5-phosphate isomerase -2.2815030940671264 -2.2462182267709454
PGM Phosphoglycerate mutase -14.716139568742836 -14.672695876996094
ENO Enolase 14.716139568742836 14.672695876996094
TALA Transaldolase 1.4969837572615654 1.4411270292601517
TKT1 Transketolase 1.4969837572615654 1.4411270292601515
TKT2 Transketolase 1.181498093245962 1.117368621884195
PPC Phosphoenolpyruvate carboxylase 2.5043094703687325 2.569978094671856
PIt2r Phosphate reversible transport via symport 3.214